In [1]:
import torch
import torchvision
import numpy as np
import os
import torch.nn as nn
import torch.optim as optim
import shutil
from approx_attributes import *
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from captum.attr import InputXGradient,IntegratedGradients,Saliency
print(torch.__version__)
print(torchvision.__version__)

2.8.0+rocm6.4
0.23.0+rocm6.4


/home/benomin/PycharmProjects/BA-Approoximation-Feature-Attribution/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = transforms.Compose([transforms.ToTensor()])

cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(cifar_testset, batch_size=1, shuffle=False)

dataset = test_loader.dataset
cifar10_labels = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

# Wähle einen zufälligen Index
idx = np.random.randint(0, len(dataset) - 1)

# Hole das Bild und Label
img, label = dataset[idx]

Using cache found in /home/benomin/.cache/torch/hub/pytorch_vision_v0.10.0
/home/benomin/PycharmProjects/BA-Approoximation-Feature-Attribution/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/benomin/PycharmProjects/BA-Approoximation-Feature-Attribution/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:

#for param in model.parameters():
#    param.requires_grad = False

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # ersetzt das MaxPooling durch "nichts"

num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

"""
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
"""

Epoch [1/20], Loss: 2.7972
Epoch [2/20], Loss: 2.5278
Epoch [3/20], Loss: 2.4017
Epoch [4/20], Loss: 2.3067
Epoch [5/20], Loss: 2.2546
Epoch [6/20], Loss: 2.2181
Epoch [7/20], Loss: 2.1952
Epoch [8/20], Loss: 2.1647
Epoch [9/20], Loss: 2.1484
Epoch [10/20], Loss: 2.1423
Epoch [11/20], Loss: 2.1220
Epoch [12/20], Loss: 2.1104
Epoch [13/20], Loss: 2.0965
Epoch [14/20], Loss: 2.0926
Epoch [15/20], Loss: 2.0859
Epoch [16/20], Loss: 2.0818
Epoch [17/20], Loss: 2.0803
Epoch [18/20], Loss: 2.0701
Epoch [19/20], Loss: 2.0729
Epoch [20/20], Loss: 2.0585


'\nnum_classes = 10\nmodel.fc = nn.Linear(model.fc.in_features, num_classes)\n\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model.parameters(), lr=0.001)\n\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel = model.to(device)\n\nnum_epochs = 5\n\nfor epoch in range(num_epochs):\n    model.train()\n    running_loss = 0.0\n    for images, labels in train_loader:\n        images, labels = images.to(device), labels.to(device)\n\n        optimizer.zero_grad()\n        outputs = model(images)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n\n        running_loss += loss.item()\n\n    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")\n'

In [7]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 15.61%


In [8]:
torch.save(model, "resnet152_cifar10_32x32_epoch20.pth")

# Laden
"""
model = torch.load("resnet18_cifar10_full.pth")
model.to(device)
model.eval()
"""

'\nmodel = torch.load("resnet18_cifar10_full.pth")\nmodel.to(device)\nmodel.eval()\n'